In [ ]:
import pandas as pd
import numpy as np

In [ ]:
data = pd.read_csv("/content/TripAdvisor_RestauarantRecommendation (1).csv")

In [ ]:
data.head()

,Name,Street Address,Location,Type,Reviews,No of Reviews,Comments,Contact Number,Trip_advisor Url,Menu,Price_Range
0,Betty Lou's Seafood and Grill,318 Columbus Ave,"San Francisco, CA 94133-3908","Seafood, Vegetarian Friendly, Vegan Options",4.5 of 5 bubbles,243 reviews,NaN,+1 415-757-0569,https://www.tripadvisor.com//Restaurant_Review...,Check The Website for a Menu,$$ - $$$
1,Coach House Diner,55 State Rt 4,"Hackensack, NJ 07601-6337","Diner, American, Vegetarian Friendly",4 of 5 bubbles,84 reviews,"Both times we were there very late, after 11 P...",+1 201-488-4999,https://www.tripadvisor.com//Restaurant_Review...,Check The Website for a Menu,$$ - $$$
2,Table Talk Diner,2521 South Rd Ste C,"Poughkeepsie, NY 12601-5476","American, Diner, Vegetarian Friendly",4 of 5 bubbles,256 reviews,Waitress was very friendly but a little pricey...,+1 845-849-2839,https://www.tripadvisor.com//Restaurant_Review...,http://tabletalkdiner.com/menu/breakfast/,$$ - $$$
3,Sixty Vines,3701 Dallas Pkwy,"Plano, TX 75093-7777","American, Wine Bar, Vegetarian Friendly",4.5 of 5 bubbles,235 reviews,Not sure why I went there for the second time....,+1 469-620-8463,https://www.tripadvisor.com//Restaurant_Review...,https://sixtyvines.com/menu/plano-tx/,$$ - $$$
4,The Clam Bar,3914 Brewerton Rd,"Syracuse, NY 13212","American, Bar, Seafood",4 of 5 bubbles,285 reviews,Doesn't look like much from the outside but wa...,+1 315-458-1662,https://www.tripadvisor.com//Restaurant_Review...,Check The Website for a Menu,$$ - $$$


In [ ]:
data.shape

(3062, 11)

In [ ]:
data.columns

Index(['Name', 'Street Address', 'Location', 'Type', 'Reviews',
       'No of Reviews', 'Comments', 'Contact Number', 'Trip_advisor Url',
       'Menu', 'Price_Range'],
      dtype='object')

In [ ]:
data["Comments"]

0                                                     NaN
1       Both times we were there very late, after 11 P...
2       Waitress was very friendly but a little pricey...
3       Not sure why I went there for the second time....
4       Doesn't look like much from the outside but wa...
                              ...                        
3057    We had to wait a few minutes to get it but it ...
3058    Came here to see Andrea Gibson perform, we wer...
3059    We come to Brio once a month. They are good fo...
3060    Stopped in yesterday for Breakfast. When in Je...
3061    First time for dinner. 2 couples. Delicious mu...
Name: Comments, Length: 3062, dtype: object

In [ ]:
data["Comments"].nunique()

2291

In [ ]:
data["Comments"][6]

'This is an easy any time place to have dinner!  From week night to a special occasion.  The food is reasonably priced and always delicious!'

In [ ]:
data['Comments'].isna().sum()

615

In [ ]:
data['Name'].nunique()

2641

In [ ]:
data["Name"].isna().sum()

0

In [ ]:
data['Location'].nunique()

2653

In [ ]:
data['Type'][10]

' Italian, Vegetarian Friendly, Gluten Free Options'

In [ ]:
data["Comments"] = data['Comments'].fillna("")

In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer

In [ ]:
tfidf = TfidfVectorizer(stop_words="english",max_features = 10000)

In [ ]:
tfidf_matrix = tfidf.fit_transform(data['Comments'])

In [ ]:
tfidf_matrix.shape

(3062, 6245)

In [ ]:
from sklearn.metrics.pairwise import linear_kernel

In [ ]:
cosine_sim = linear_kernel(tfidf_matrix, tfidf_matrix)

In [ ]:
indices = pd.Series(data.index,index=data['Comments'])

In [ ]:
indices

Comments
                                                                                                                                                                                                                                                                                                                      0
Both times we were there very late, after 11 PM.  At that time in many diners (forget restaurants!) you get warmed-over food and lousy service.  Not so here - food was uniformly very good and the service quite good.  There weren't many people but it...More                                                      1
Waitress was very friendly but a little pricey for a diner but the food was delicious \nThe menu had a wide variety of foods to choose from but any choice would have been a good one                                                                                                                                 2
Not sure why I went there for the second time. Will not

In [ ]:
from sklearn.metrics.pairwise import cosine_similarity

In [ ]:
similarity = cosine_similarity(tfidf_matrix, tfidf_matrix)

In [ ]:
def content_recommender(Comments, cosine_sim=cosine_sim, data=data, indices=indices):

    index = indices[Comments]
    sim_scores = list(enumerate(cosine_sim[index]))
    sim_scores = sorted(sim_scores, key=lambda x: x[1], reverse=True)
    sim_scores = sim_scores[1:11]
    comments_indices = [i[0] for i in sim_scores]
    return data['Comments'].iloc[comments_indices]

In [ ]:
content_recommender("This is an easy any time place to have dinner!  From week night to a special occasion.  The food is reasonably priced and always delicious!")

1648    I was staying at the Super 8 next door.  Apple...
801     Great place to celebrate an occasion or just t...
1498    Great place to celebrate an occasion or just t...
802     Very large portions of all items, including su...
2970    The place is nicely decorated with cozy atmosp...
2379    We have been to both the South Lake Union and ...
660     2nd time eating here.  Service this time was e...
2704    Jose Tejas is a great place to go to for lunch...
2431    Too noisy. Hard to visit. Expensive. Food was ...
2273    The menu was far too limited.  Hoping this was...
Name: Comments, dtype: object